In [1]:
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

import pandas as pd
import numpy as np
np.random.seed(0)

from tqdm.notebook import tqdm

import os

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train= pd.read_csv('data/train_onehot.csv',index_col=0)
test= pd.read_csv('data/test_onehot.csv',index_col=0)
submission = pd.read_csv('data/sample_submission.csv')

In [3]:
data=pd.concat([train, test], axis=0)
data

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_missing_job,occyp_type_unknown
0,0,202500.0,-13899,-4709,1,0,0,0,2.0,-6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,247500.0,-11380,-1540,1,0,0,1,3.0,-5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,450000.0,-19087,-4434,1,0,1,0,2.0,-22.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,202500.0,-15088,-2092,1,0,1,0,2.0,-37.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0,157500.0,-15037,-2105,1,0,0,0,2.0,-26.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.0,-18593,-5434,1,1,1,0,2.0,-19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0,202500.0,-10886,-1315,1,1,0,0,2.0,-34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0,292500.0,-21016,-14018,1,0,0,0,2.0,-55.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0,180000.0,-16541,-1085,1,0,1,0,2.0,-33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
def days_to_age(x):
    return abs(x)/365
data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(days_to_age)

In [5]:
def minus(x):
    return x * -1
data['begin_month'] = data['begin_month'].apply(minus)

In [6]:
data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(days_to_age)

In [7]:
data.loc[data['child_num'] >= 2,'child_num']

5       2
15      2
39      2
41      2
53      2
       ..
9927    2
9964    4
9973    2
9977    2
9985    2
Name: child_num, Length: 3764, dtype: int64

In [8]:
data.tail(40)

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_missing_job,occyp_type_unknown
9960,0,180000.0,59.408219,6.635616,1,0,0,0,2.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9961,0,184500.0,54.454795,1.367123,1,0,0,0,2.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9962,0,135000.0,41.380822,4.654795,1,0,0,0,2.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9963,0,157500.0,36.016438,5.926027,1,0,0,0,2.0,59.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9964,4,157500.0,31.178082,4.750685,1,0,0,0,6.0,11.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9965,0,270000.0,51.120548,1.115068,1,0,0,0,1.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9966,1,135000.0,27.753425,4.523288,1,0,0,0,2.0,57.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9967,0,270000.0,59.893151,0.000000,1,0,0,0,1.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9968,0,135000.0,55.210959,0.000000,1,0,1,0,2.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9969,0,94500.0,51.268493,9.936986,1,1,0,0,2.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36457 entries, 0 to 9999
Data columns (total 58 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   child_num                               36457 non-null  int64  
 1   income_total                            36457 non-null  float64
 2   DAYS_BIRTH                              36457 non-null  float64
 3   DAYS_EMPLOYED                           36457 non-null  float64
 4   FLAG_MOBIL                              36457 non-null  int64  
 5   work_phone                              36457 non-null  int64  
 6   phone                                   36457 non-null  int64  
 7   email                                   36457 non-null  int64  
 8   family_size                             36457 non-null  float64
 9   begin_month                             36457 non-null  float64
 10  credit                                  26457 non-null  flo

In [9]:
train=data[:len(data)-10000]
test=data[len(data)-10000:]

In [10]:
test

,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,...,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,occyp_type_missing_job,occyp_type_unknown
0,0,112500.0,60.246575,0.000000,1,0,1,0,2.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,135000.0,51.956164,23.756164,1,0,1,0,2.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,69372.0,43.526027,0.594521,1,1,1,0,2.0,40.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,112500.0,52.794521,6.934247,1,1,0,0,2.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,225000.0,48.827397,25.712329,1,1,0,0,2.0,8.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,202500.0,50.939726,14.887671,1,1,1,0,2.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,0,202500.0,29.824658,3.602740,1,1,0,0,2.0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0,292500.0,57.578082,38.405479,1,0,0,0,2.0,55.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9998,0,180000.0,45.317808,2.972603,1,0,1,0,2.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test.drop('credit', axis=1)

In [12]:
np.random.seed(42)
if "Set" not in train.columns:
    train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))

train_indices = train_x[train.Set=="train"].index
valid_indices = train_x[train.Set=="valid"].index

<ipython-input-12-c19fed2c6b6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(train.shape[0],))


In [13]:
train.nunique()

child_num                                    9
income_total                               249
DAYS_BIRTH                                6621
DAYS_EMPLOYED                             3470
FLAG_MOBIL                                   1
work_phone                                   2
phone                                        2
email                                        2
family_size                                 10
begin_month                                 61
credit                                       3
gender_F                                     2
gender_M                                     2
car_N                                        2
car_Y                                        2
reality_N                                    2
reality_Y                                    2
income_type_Commercial associate             2
income_type_Pensioner                        2
income_type_State servant                    2
income_type_Student                          2
income_type_W

In [14]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in tqdm(train_x.columns):
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train_x[col].nunique())
        l_enc = LabelEncoder()
        train_x[col] = train_x[col].fillna("NaN")
        train_x[col] = l_enc.fit_transform(train_x[col].values)
        try:
            test_X[col] = test_x[col].fillna("NaN")
            test_x[col] = l_enc.transform(test_x[col].values)
        except:
            print(f"Column {col} does not exist in test set")
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        training_mean = train_x.loc[train_indices, col].mean()
        train.fillna(training_mean, inplace=True)
        test.fillna(training_mean, inplace=True)

C:\Users\82105\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


child_num 9
Column child_num does not exist in test set
FLAG_MOBIL 1
Column FLAG_MOBIL does not exist in test set
work_phone 2
Column work_phone does not exist in test set
phone 2
Column phone does not exist in test set
email 2
Column email does not exist in test set
family_size 10
Column family_size does not exist in test set
begin_month 61
Column begin_month does not exist in test set
gender_F 2
Column gender_F does not exist in test set
gender_M 2
Column gender_M does not exist in test set
car_N 2
Column car_N does not exist in test set
car_Y 2
Column car_Y does not exist in test set
reality_N 2
Column reality_N does not exist in test set
reality_Y 2
Column reality_Y does not exist in test set
income_type_Commercial associate 2
Column income_type_Commercial associate does not exist in test set
income_type_Pensioner 2
Column income_type_Pensioner does not exist in test set
income_type_State servant 2
Column income_type_State servant does not exist in test set
income_type_Student 2
Co

In [15]:
features = [ col for col in train_x.columns] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [16]:
X_train = train_x[features].values[train_indices]
y_train = train_y.values[train_indices]

X_valid = train_x[features].values[valid_indices]
y_valid = train_y.values[valid_indices]

X_test = test_x[features].values

In [17]:
clf = TabNetMultiTaskClassifier(n_steps=1,
                                cat_idxs=cat_idxs,
                                cat_dims=cat_dims,
                                cat_emb_dim=1,
                                optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":50,
                                                  "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax', 
                                lambda_sparse=0, 
                       
                      )

Device used : cpu


In [18]:
max_epochs = 250
clf.fit(
    X_train=X_train, y_train=y_train,
    max_epochs=max_epochs ,
    patience=50, 
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)

No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.9445  |  0:00:04s
epoch 1  | loss: 0.88284 |  0:00:09s
epoch 2  | loss: 0.87801 |  0:00:14s
epoch 3  | loss: 0.87223 |  0:00:19s
epoch 4  | loss: 0.86725 |  0:00:24s
epoch 5  | loss: 0.86238 |  0:00:29s
epoch 6  | loss: 0.85905 |  0:00:34s
epoch 7  | loss: 0.84909 |  0:00:39s
epoch 8  | loss: 0.83519 |  0:00:44s
epoch 9  | loss: 0.80937 |  0:00:49s
epoch 10 | loss: 0.79426 |  0:00:54s
epoch 11 | loss: 0.78794 |  0:00:59s
epoch 12 | loss: 0.78765 |  0:01:04s
epoch 13 | loss: 0.78211 |  0:01:09s
epoch 14 | loss: 0.77723 |  0:01:14s
epoch 15 | loss: 0.77565 |  0:01:19s
epoch 16 | loss: 0.77293 |  0:01:24s
epoch 17 | loss: 0.76883 |  0:01:29s
epoch 18 | loss: 0.76867 |  0:01:34s
epoch 19 | loss: 0.76461 |  0:01:39s
epoch 20 | loss: 0.76271 |  0:01:44s
epoch 21 | loss: 0.75941 |  0:01:49s
epoch 22 | loss: 0.75687 |  0:01:54s
epoch 23 | loss: 0.75509 |  0:01:59s
epoch 24 | loss: 0.75265 |  0:02:04s
ep

epoch 220| loss: 0.64066 |  0:19:31s
epoch 221| loss: 0.63939 |  0:19:36s
epoch 222| loss: 0.63689 |  0:19:41s
epoch 223| loss: 0.64052 |  0:19:46s
epoch 224| loss: 0.64032 |  0:19:51s
epoch 225| loss: 0.64417 |  0:19:56s
epoch 226| loss: 0.6398  |  0:20:01s
epoch 227| loss: 0.63808 |  0:20:06s
epoch 228| loss: 0.64189 |  0:20:11s
epoch 229| loss: 0.64224 |  0:20:16s
epoch 230| loss: 0.64268 |  0:20:21s
epoch 231| loss: 0.63912 |  0:20:26s
epoch 232| loss: 0.63672 |  0:20:31s
epoch 233| loss: 0.63989 |  0:20:36s
epoch 234| loss: 0.64026 |  0:20:41s
epoch 235| loss: 0.63967 |  0:20:46s
epoch 236| loss: 0.63371 |  0:20:51s
epoch 237| loss: 0.64057 |  0:20:56s
epoch 238| loss: 0.63766 |  0:21:01s
epoch 239| loss: 0.63788 |  0:21:06s
epoch 240| loss: 0.64394 |  0:21:10s
epoch 241| loss: 0.64495 |  0:21:15s
epoch 242| loss: 0.63765 |  0:21:20s
epoch 243| loss: 0.63977 |  0:21:25s
epoch 244| loss: 0.64293 |  0:21:36s
epoch 245| loss: 0.63806 |  0:21:47s
epoch 246| loss: 0.63916 |  0:21:52s
e

In [19]:
preds_valid = clf.predict_proba(X_valid) 

In [20]:
preds = clf.predict_proba(X_test)

IndexError: index out of range in self

In [22]:
clf.feature_importances_

array([4.19621024e-03, 6.96794064e-03, 1.10236948e-03, 1.98193889e-04,
       5.44890484e-03, 5.97199736e-03, 2.65047967e-02, 1.35498302e-04,
       7.71926226e-03, 6.62735410e-02, 8.02804385e-03, 1.36867202e-01,
       8.99356810e-02, 3.00843828e-02, 1.15496382e-01, 6.27760002e-03,
       2.71165713e-02, 4.28815912e-04, 5.88426062e-03, 2.31055193e-04,
       1.17604376e-04, 2.10982868e-05, 1.87200784e-01, 3.23235277e-04,
       0.00000000e+00, 1.02880017e-01, 9.55648674e-03, 0.00000000e+00,
       0.00000000e+00, 5.47262454e-03, 0.00000000e+00, 1.50582016e-02,
       1.57746047e-02, 1.35686424e-03, 1.88144081e-03, 1.05801635e-02,
       7.34160997e-04, 2.46885449e-02, 1.83564589e-03, 8.28503931e-04,
       1.63649476e-04, 4.82008393e-05, 1.69783614e-02, 3.47283590e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.27898653e-04,
       0.00000000e+00, 3.95918242e-06, 5.37411336e-03, 8.87359513e-03,
       7.36204468e-04, 9.07157441e-03, 4.02190171e-04, 3.13203911e-04,
      

In [23]:
i, j, k = [], [], []
for row in range(10000):
  i.append(preds[0][row][0])
  j.append(preds[0][row][1])
  k.append(preds[0][row][2])

len(i), len(j), len(k)

NameError: name 'preds' is not defined

In [24]:
X_test

array([[0.00000000e+00, 1.12500000e+05, 6.02465753e+01, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.35000000e+05, 5.19561644e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.93720000e+04, 4.35260274e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.92500000e+05, 5.75780822e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.80000000e+05, 4.53178082e+01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 2.70000000e+05, 2.50794521e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])